# Парсинг данных 

**Web парсинг** -  способ считывания различных данных, расположенных на веб-страницах, для их систематизации и дальнейшего анализа.

Открытые данные часто публикуют в форматах (таких как csv, json, xml), уобоных для загрузки в программы анализа, например: https://www.nalog.gov.ru/opendata/

Но существуют данные, даже открытые, которые доступны только в виде отдельных веб-страниц. Сбор данных с таких страниц становится отдельной задачей. Например:
https://www.gov.uk/find-out-if-a-company-is-in-financial-trouble

Процедуру Web парсинга можно разбить на два этапа:

1. отправка запроса на web-сайт и загрузка исходного кода страницы;
2. извлечение содержимого web-страницы. 


# Библиотека [Beautiful Soup](<https://www.crummy.com/software/BeautifulSoup/bs4/doc/>)


## Тестовый HTML
Намного проще чем реальные веб-страницы.

In [ ]:
test = '''
    <html>
        <head><title>Some title</title></head>
        <body>
            <div class="content-box" id="main">
                <h2 align='center'> Some text </h2>
                <h2 align='left'> Another text </h2>
            </div>
            <h2> Last <b>text</b> </h2>
        </body>
    </html>
'''

### Создание объекта `BeautifulSoup`

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(test, 'html.parser')
soup

### Базовые команды в Beautiful Soup для разбора HTML документа

In [ ]:
# тег как объект
print(soup.title)
print(type(soup.title))

In [ ]:
# str тега - просто напрямую его текст
soup.div

In [ ]:
# тег можно спросить - какой тег?
soup.title.name

In [ ]:
# текстовое содержимое тега
soup.title.string

In [ ]:
soup.h2

In [ ]:
# родитель тега
soup.title.parent

In [ ]:
# дети тега
print(type(soup.div.children))
list(soup.div.children)

In [ ]:
type(list(soup.div.children)[0])

In [ ]:
# аттрибуты тега доступны по имени как по ключу
print(soup.div.id)
soup.div['id']

Что если одинаковых тегов несколько?

In [ ]:
soup.div.h2

In [ ]:
soup.h2

In [ ]:
soup.find_all('h2')

In [ ]:
# find_all для удобства можно опостить - просто вызывая тег как функцию
soup('h2')

In [ ]:
# какие объекты есть в BeautifulSoup?
type(soup), type(soup.title), type(soup.div.children), type(soup.title.string), type(soup('h2'))

#### 1. найти h2 теги с атрибутом align равным center

In [ ]:
soup("h2", align='center')

#### 2. найти h2 теги у которых есть аттрибут align

In [ ]:
soup('h2', align=True)

#### 3. извлечь текстовое содержимое тега с разметкой внутри

In [ ]:
tag = soup('h2')[2]
tag

In [ ]:
print(list(tag.children))

In [ ]:
print([c.string for c in tag.children])
print(''.join([c.string for c in tag.children]))

In [ ]:
print(tag.get_text())

In [ ]:
soup.body

In [ ]:
print([c.string for c in soup.body.children])

In [ ]:
print(soup.body.string)

In [ ]:
soup.body.get_text()

# Рассмотрим реальный HTML документ
В качестве примера возьмем статью из Википедии:  
<https://ru.wikipedia.org/wiki/Дубна>

## Протокол HTTP: библиотека [requests](https://2.python-requests.org/en/latest/)

### Отправляем GET запрос на web-сервер и загружаем HTML

In [ ]:
import requests
r = requests.get('https://ru.wikipedia.org/wiki/Дубна')
r

In [ ]:
r.status_code

In [ ]:
print(type(r.content))

In [ ]:
r.content[:1000]

In [ ]:
r.text

### Парсим полученный HTML

In [ ]:
dubna = BeautifulSoup(r.content, 'html.parser')

In [ ]:
print(dubna.prettify()[:10000])

#### Рассмотрим верхний уровень структуры реальной страницы

In [ ]:
dubna.children

In [ ]:
[type(item) for item in list(dubna.children)]

In [ ]:
list(dubna.children)[0]

In [ ]:
list(dubna.children)[1]

объект `Doctype` содержит информацию о типе документа

объект `NavigableString` - текст, найденный в документе (между doctype и `<html>`

объект `Tag` - тег `<html>...</html>`

In [ ]:
html = list(dubna.children)[2]
html is dubna.html # то же самое, ведь тег html в документе - один

In [ ]:
len(list(dubna.html.children))

In [ ]:
from bs4 import NavigableString
[
    (
        type(item),
        item if type(item) is NavigableString else item.name
    )
    for item in list(dubna.html.children)
]

In [ ]:
body = list(html.children)[3]
body is html.body  # тот же тег, ведь body в документе один

In [ ]:
[
    (type(item), item)
    if type(item) is NavigableString else 
    (type(item), item.name, item.get('id'), item.get('class'))
    for item in list(dubna.body.children)
]

#### Извлекаем осноовной текст из статьи

In [ ]:
content = dubna.find("div", id='mw-content-text').find("div", class_='mw-parser-output')
print(content.get_text())

In [ ]:
first_paragraphs = content('p', recursive=False)[:2]
first_paragraphs

In [ ]:
from pprint import pprint
main_info_str = '\n'.join([p.get_text() for p in first_paragraphs])
pprint(main_info_str)

#### Список районов

In [ ]:
content.find(text="Внутреннее деление")

In [ ]:
content.find(text="Внутреннее деление").parent

In [ ]:
cell = content.find(text="Внутреннее деление").parent.find_next_sibling('td')
cell

In [ ]:
cell('span')

In [ ]:
districts = [s.get_text().strip(',') for s in cell('span')]
districts

#### Таблица климата

In [ ]:
climate_data = content.find(id='Климат').find_next('table')
climate_data

In [ ]:
climate_data.tr

In [ ]:
columns = [th.get_text().strip() for th in climate_data.tr('th')]
columns

In [ ]:
import pandas as pd

values = []
for tr in climate_data('tr')[1:-1]:
    td = tr(['th', 'td'])
    row = [tr.text.strip() for tr in td]
    values.append(row)

climate = pd.DataFrame(values, columns=columns)
climate

In [ ]:
climate_t = climate.set_index('Показатель').T
climate_t.index.name = 'Период'
climate_t

#### Изображения

In [ ]:
thumbs = dubna('div', class_='thumb')
len(thumbs)

In [ ]:
srcs = [div.img['src'] for div in thumbs]
srcs

In [ ]:
links = ['https:' + src for src in srcs]
links

In [ ]:
fnames = [link.split('/')[-1] for link in links]
fnames

In [ ]:
cd /content

In [ ]:
mkdir -p /content/dubna_images

In [ ]:
cd dubna_images

In [ ]:
ls

In [ ]:
def dwnld_image(url, save_to):
    response = requests.get(url, 
                            allow_redirects=True,
                            headers = 
                            {'User-Agent': f'StudentCode/0.1 requests/{requests.__version__}'}
    )
    print(url, response)
    if response.status_code != 200:
        raise Exception(response.text)
        
    with open(save_to, 'wb') as f:        
        f.write(response.content)

In [ ]:
from time import sleep
for l, fn in zip(links, fnames):
    dwnld_image(l, '/content/dubna_images/' + fn)
    sleep(3)

In [ ]:
from IPython.display import Image
from pathlib import Path
for fname in Path('/content/dubna_images').glob('*'):
    print(fname)
    display(Image(filename=fname))


In [ ]:
from skimage.io import imread_collection

col = imread_collection('/content/dubna_images/*')

In [ ]:
import matplotlib.pyplot as plt

for img in col:
    plt.figure()
    plt.imshow(img)
    plt.show()

# Библитотека LXML - еще одна популярная библиотека парсинга HTML, но в первую очередь XML.

Документация подробна, но достаточно сложная для понимания. Например
есть два варианта парсинга html:
*   https://lxml.de/parsing.html#parsing-html
*   https://lxml.de/lxmlhtml.html#parsing-html

In [ ]:
from lxml import etree
xml_tree = etree.HTML(test)
xml_tree

In [ ]:
from lxml.html import fromstring, parse
html_tree = fromstring(test)
html_tree

In [ ]:
type(xml_tree), type(html_tree)

lxml.html - методы специфические для html, лучше использовать его.

# Задание

Вместо Beautiful Soup использовать библиотеку [LXML](https://lxml.de/).

1. найти h2 теги с атрибутом align равным center
2. найти h2 теги у которых есть аттрибут align
3. извлечь текстовое содержимое тега с разметкой внутри
4. извлечь два первых параграфа с текстом из страницы на Википедии
5. извлечь список районов Дубны
6. извлечь таблицу климата
7. сохранить изображения